In [ ]:
import pandas as pd
import statsmodels.api as sm
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

data = pd.read_csv('imdb_movie_data.csv')
df = pd.DataFrame(data)

print(df.info())
df

#Fixing data to give better results
df['gross_revenue (million USD)'] = np.log(df['gross_revenue (million USD)'])
# df['vote_count'] = np.log(df['vote_count'])
# df['user_reviews'] = np.log(df['user_reviews'])
# df['runtime'] = np.log(df['runtime'])
lower = df['gross_revenue (million USD)'].quantile(0.10)
upper = df['gross_revenue (million USD)'].quantile(0.90)

# Select rows where gross_revenue is between the 10th and 90th percentiles
df = df[(df['gross_revenue (million USD)'] >= lower) & (df['gross_revenue (million USD)'] <= upper)]

#OLS Regression on the entire data
#winter and unrated are baselines for dummy variables so they are omitted

# print(df[['runtime', 'user_reviews', 'star_rating', 'vote_count', 'spring', 'summer', 'fall', 'G', 'PG', 'PG-13', 'R']].var())
X = df[['runtime', 'user_reviews', 'star_rating', 'vote_count', 'spring', 'summer', 'fall', 'G', 'PG', 'PG-13', 'R']]
y = df['gross_revenue (million USD)']
X = sm.add_constant(X)
ols_model = sm.OLS(y, X).fit()
print(ols_model.summary())

# Multicolinearity test of numeric variables
X = df[['runtime', 'user_reviews', 'star_rating', 'vote_count', 'summer', 'fall', 'winter', 'G', 'PG', 'PG-13', 'R']]

X = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

#Heteroskedasticity test
bp_test = het_breuschpagan(ols_model.resid, ols_model.model.exog)

bp_test_statistic = bp_test[0]
bp_p_value = bp_test[1]

print(f"Breusch-Pagan test statistic: {bp_test_statistic}")
print(f"Breusch-Pagan p-value: {bp_p_value}")

#Define features and target variables
features = ['runtime', 'user_reviews', 'star_rating', 'vote_count', 'spring', 'summer', 'fall', 'winter', 'G', 'PG', 'PG-13','Unrated']
target = 'gross_revenue (million USD)'

X = df[features]
y = df[target]

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

#OLS
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test) 

#Fit using training set
ols_model = sm.OLS(y_train, X_train).fit()

#Make predictions using the model
y_pred_ols = ols_model.predict(X_test)

#Evaluate model performance using R-score and mean squared error (MSE) between predicted and actual values
r_squared_ols = r2_score(y_test, y_pred_ols)
print(f'R-squared (OLS): {r_squared_ols}')

mse_ols = mean_squared_error(y_test, y_pred_ols)
print(f'Mean Squared Error (OLS): {mse_ols}')

#Random Forest
rf_model = RandomForestRegressor(n_estimators=500)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Evaluate random forest model
r_squared_rf = r2_score(y_test, y_pred_rf)
print(f'R-squared (RF): {r_squared_rf}')

mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Mean Squared Error (RF): {mse_rf}')

#Decision Tree
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

# Evaluate decision tree model
r_squared_tree = r2_score(y_test, y_pred_tree)
print(f'R-squared (Tree): {r_squared_tree}')

mse_tree = mean_squared_error(y_test, y_pred_tree)
print(f'Mean Squared Error (Tree): {mse_tree}')

#Gradient Boosting Algorithm
gb_model = GradientBoostingRegressor(n_estimators=100)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

# Evaluate GB model
r_squared_gb = r2_score(y_test, y_pred_gb)
print(f'R-squared (GB): {r_squared_gb}')

mse_gb = mean_squared_error(y_test, y_pred_gb)
print(f'Mean Squared Error (GB): {mse_gb}')

#Neural Network
nn_model = MLPRegressor(hidden_layer_sizes=(100), max_iter=1000)
nn_model.fit(X_train, y_train)
y_pred_nn = nn_model.predict(X_test)

# Evaluate NN model
r_squared_nn = r2_score(y_test, y_pred_nn)
print(f'R-squared (NN): {r_squared_nn}')

mse_nn = mean_squared_error(y_test, y_pred_nn)
print(f'Mean Squared Error (NN): {mse_nn}')

#Light GBM
lgb_model = lgb.LGBMRegressor(n_estimators=50, verbosity = -1)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)

r_squared_lgb = r2_score(y_test, y_pred_lgb)
print(f'R-squared (LGB): {r_squared_lgb}')

mse_lgb = mean_squared_error(y_test, y_pred_lgb)
print(f'Mean Squared Error (LGB): {mse_lgb}')

models = ['OLS', 'RF', 'Tree', 'GB', 'NN', 'LGB']
r_squared = [r_squared_ols, r_squared_rf, r_squared_tree, r_squared_gb, r_squared_nn, r_squared_lgb]
mse = [mse_ols, mse_rf, mse_tree, mse_gb, mse_nn, mse_lgb]

metrics_df = pd.DataFrame({'Model Type': models, 'R-squared': r_squared, 'MSE': mse})
plt.figure(figsize=(12, 6))

# Create a bar plot for R-squared
plt.subplot(1, 2, 1)
sns.barplot(x='Model Type', y='R-squared', data=metrics_df)
plt.title('R-squared for Different Models')
plt.ylim(0, 1)
plt.ylabel('R-squared')

# Create a bar plot for MSE
plt.subplot(1, 2, 2)
sns.barplot(x='Model Type', y='MSE', data=metrics_df)
plt.title('Mean Squared Error for Different Models')
plt.ylabel('Mean Squared Error')

plt.tight_layout()
plt.show()

residuals_df = pd.DataFrame({
    'OLS': y_test - y_pred_ols,
    'Random Forest': y_test - y_pred_rf,
    'Decision Tree': y_test - y_pred_tree,
    'Gradient Boosting': y_test - y_pred_gb,
    'Neural Network': y_test - y_pred_nn,
    'Light Gradient Boosting': y_test - y_pred_lgb
})

plt.figure(figsize=(12, 8))
for i, model in enumerate(residuals_df.columns):
    plt.subplot(3, 2, i + 1)
    sns.scatterplot(x=y_test, y=residuals_df[model], alpha=0.6)
    plt.axhline(0, color='red', linestyle='--')
    plt.title(f'Residuals for {model}')
    plt.xlabel('log(Gross Box Office Revenue)')
    plt.ylabel('Residual (%)')

plt.tight_layout()
plt.show()